In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
i

In [ ]:
path = './dataset/train_images/' # Path to directory which contains classes
classes = os.listdir(path) # List of all classes
print('Total number of categories: {0}'.format(len(classes)))

In [ ]:
counts = {}
for c in classes:
    counts[c] = len(os.listdir(os.path.join(path, c)))
    
print('Total number of images in dataset: {0}'.format(sum(list(counts.values()))))

# Number of images in each clsss plot
fig = plt.figure(figsize = (25, 5))
sns.barplot(x = list(counts.keys()), y = list(counts.values())).set_title('Number of images in each class')
plt.xticks(rotation = 90)
plt.margins(x=0)
plt.show()

In [ ]:
X = [] # List for images
Y = [] # List for labels
label_to_name = [0 for i in range(len(classes))]
# Loop through all classes
for label, c in enumerate(classes):
    # Reading, resizing and adding image and label to lists
    label_to_name[label] = c
    dir_path = os.path.join(path, c)
    for i in os.listdir(dir_path):
        
        image = cv2.imread(os.path.join(dir_path, i))

        X.append(image)
        Y.append(label)  

print('DONE')

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
X = X / 255.0

In [ ]:
plt.imshow(X[1000])

In [ ]:
label_to_name[Y[1000]]

In [ ]:
model = tf.keras.applications.Xception(input_shape=(300,300,3), classes=14, weights=None)

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X, Y, epochs=2, verbose=True)